In [ ]:
# =============================================================================
# Logis BRITO - SEMANAS 1 & 2: DEFINIÇÃO DA ONTOLOGIA DE CONFLITO URBANO
# =============================================================================

# 1. SETUP INICIAL: IMPORTS E DEFINIÇÃO DOS "DICIONÁRIOS" (NAMESPACES)
# -----------------------------------------------------------------------------
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, OWL, XSD

# Inicializa o Grafo de Conhecimento
g = Graph()

# Define nosso vocabulário principal. Tudo relacionado ao nosso modelo de conflito
# usará o prefixo "rec".
REC = Namespace("http://recife.leg.br/ontologia-conflito#")

# Associa os prefixos aos namespaces para um arquivo de saída mais limpo
g.bind("rec", REC)
g.bind("owl", OWL)
g.bind("rdfs", RDFS)

# =============================================================================
# 2. DEFINIÇÃO DAS CLASSES: OS "ATORES" E "CONCEITOS" DO JOGO URBANO
# Hierarquia baseada nos eixos críticos da narrativa.
# =============================================================================

# --- Eixo 1: Veto & Inação (As Causas Raiz do Problema) ---
VetoInacao = REC.VetoInacao
g.add((VetoInacao, RDF.type, OWL.Class))
g.add((VetoInacao, RDFS.label, Literal("Veto & Inaction")))
g.add((VetoInacao, RDFS.comment, Literal("A classe de todas as ações ou omissões que impedem o desenvolvimento urbano justo.")))

Estado_Inativo = REC.Estado_Inativo
g.add((Estado_Inativo, RDF.type, OWL.Class))
g.add((Estado_Inativo, RDFS.subClassOf, VetoInacao))
g.add((Estado_Inativo, RDFS.label, Literal("Estado Inativo")))

Elite_Proprietaria = REC.Elite_Proprietaria
g.add((Elite_Proprietaria, RDF.type, OWL.Class))
g.add((Elite_Proprietaria, RDFS.subClassOf, VetoInacao))
g.add((Elite_Proprietaria, RDFS.label, Literal("Elite Proprietária")))

# --- Eixo 2: Instrumentos & Ação (As Ferramentas de Reversão) ---
InstrumentoAcao = REC.InstrumentoAcao
g.add((InstrumentoAcao, RDF.type, OWL.Class))
g.add((InstrumentoAcao, RDFS.label, Literal("Instrumento & Ação")))

PEUC = REC.PEUC
g.add((PEUC, RDF.type, OWL.Class))
g.add((PEUC, RDFS.subClassOf, InstrumentoAcao))
g.add((PEUC, RDFS.label, Literal("PEUC - Parcelamento, Edificação ou Utilização Compulsórios")))

OODC = REC.OODC
g.add((OODC, RDF.type, OWL.Class))
g.add((OODC, RDFS.subClassOf, InstrumentoAcao))
g.add((OODC, RDFS.label, Literal("OODC - Outorga Onerosa do Direito de Construir")))

# --- Eixo 3: Conflito & Espaço (Onde a Disputa Acontece) ---
EspacoDeConflito = REC.EspacoDeConflito
g.add((EspacoDeConflito, RDF.type, OWL.Class))
g.add((EspacoDeConflito, RDFS.label, Literal("Espaço de Conflito")))

ZEIS = REC.ZEIS
g.add((ZEIS, RDF.type, OWL.Class))
g.add((ZEIS, RDFS.subClassOf, EspacoDeConflito))
g.add((ZEIS, RDFS.label, Literal("ZEIS - Zona Especial de Interesse Social")))

Centro_Ocioso = REC.Centro_Ocioso
g.add((Centro_Ocioso, RDF.type, OWL.Class))
g.add((Centro_Ocioso, RDFS.subClassOf, EspacoDeConflito))
g.add((Centro_Ocioso, RDFS.label, Literal("Centro Ocioso")))

# --- Eixo 4: Dano & Consequência (Os Custos da Inação) ---
DanoUrbano = REC.DanoUrbano
g.add((DanoUrbano, RDF.type, OWL.Class))
g.add((DanoUrbano, RDFS.label, Literal("Dano & Consequência")))

Caos_Funcional = REC.Caos_Funcional
g.add((Caos_Funcional, RDF.type, OWL.Class))
g.add((Caos_Funcional, RDFS.subClassOf, DanoUrbano))
g.add((Caos_Funcional, RDFS.label, Literal("Caos Funcional")))

Arrecadacao_Perdida = REC.Arrecadacao_Perdida
g.add((Arrecadacao_Perdida, RDF.type, OWL.Class))
g.add((Arrecadacao_Perdida, RDFS.subClassOf, DanoUrbano))
g.add((Arrecadacao_Perdida, RDFS.label, Literal("Arrecadação Perdida")))

Doenca_e_Morte = REC.Doenca_e_Morte
g.add((Doenca_e_Morte, RDF.type, OWL.Class))
g.add((Doenca_e_Morte, RDFS.subClassOf, DanoUrbano))
g.add((Doenca_e_Morte, RDFS.label, Literal("Doença e Morte")))

# =============================================================================
# 3. DEFINIÇÃO DAS PROPRIEDADES: AS "REGRAS DO JOGO"
# Causalidade, Antagonismo e Instrumentalidade
# =============================================================================

# --- Relações de Causalidade e Reversão ---
causa_direta = REC.causa_direta
g.add((causa_direta, RDF.type, OWL.ObjectProperty))
g.add((causa_direta, RDFS.comment, Literal("Relação causal: X causa diretamente Y. (Ex: Inação causa Dano)")))

impede = REC.impede
g.add((impede, RDF.type, OWL.ObjectProperty))
g.add((impede, RDFS.comment, Literal("Relação de bloqueio: X impede a ocorrência/aplicação de Y. (Ex: Veto impede Ação)")))

reverte_por_forca = REC.reverte_por_forca
g.add((reverte_por_forca, RDF.type, OWL.ObjectProperty))
g.add((reverte_por_forca, RDFS.comment, Literal("Relação de poder: X reverte o efeito de Y. (Ex: Instrumento reverte Veto)")))

financia_a_reversao = REC.financia_a_reversao
g.add((financia_a_reversao, RDF.type, OWL.ObjectProperty))
g.add((financia_a_reversao, RDFS.comment, Literal("Relação instrumental: X gera recursos para financiar a solução Y.")))

# --- Relação de Antagonismo ---
em_antagonismo_com = REC.em_antagonismo_com
g.add((em_antagonismo_com, RDF.type, OWL.SymmetricProperty)) # Se A é antagonista de B, B é de A
g.add((em_antagonismo_com, RDFS.comment, Literal("Relação de disputa: X e Y têm objetivos conflitantes.")))

# --- Atributos de Quantificação (A Lógica Descritiva) ---
nivelDeAcao = REC.nivelDeAcao
g.add((nivelDeAcao, RDF.type, OWL.DatatypeProperty))
g.add((nivelDeAcao, RDFS.comment, Literal("Classifica a 'audácia' de um instrumento: Passivo, Punitivo_Moderado, Punitivo_Agressivo.")))

metaQuantificavel = REC.metaQuantificavel
g.add((metaQuantificavel, RDF.type, OWL.DatatypeProperty))
g.add((metaQuantificavel, RDFS.comment, Literal("Define um contrato de desempenho numérico para um instrumento.")))

# =============================================================================
# 4. SALVAR A ONTOLOGIA (O SCHEMA)
# =============================================================================

# Salva a estrutura da sua ontologia em um arquivo .ttl (Turtle)
# Este arquivo será a base para as próximas fases do projeto.
output_path = "../data/ontologia_conflito_urbano_schema.ttl"
g.serialize(destination=output_path, format="turtle")

print(f"Ontologia base criada e salva em: {output_path}")
print(f"Total de triplas (declarações lógicas) no grafo: {len(g)}")

Ontologia base criada e salva em: ../data/ontologia_conflito_urbano_schema.ttl
Total de triplas (declarações lógicas) no grafo: 50


In [2]:
# =============================================================================
# ORÁCULO DE BRITO - SEMANAS 3 & 4: INSTANCIAÇÃO DO CONFLITO
# Caso de Estudo: ZEIS Coque e a Lei de Remembramento
# =============================================================================

# (Assumindo que os imports e namespaces já foram definidos na célula anterior)

# 1. CARREGAR A ONTOLOGIA BASE (O ARQUIVO .ttl QUE VOCÊ GEROU)
# -----------------------------------------------------------------------------
g = Graph()
g.parse("../data/ontologia_conflito_urbano_schema.ttl", format="turtle")

# 2. CRIAR OS INDIVÍDUOS (AS "PEÇAS" DO JOGO)
# -----------------------------------------------------------------------------

# Atores
prefeitura = REC["Prefeitura_do_Recife"]
mercado_imobiliario_especulativo = REC["Mercado_Imobiliario_Especulativo"]
comunidade_coque = REC["Comunidade_do_Coque"]

# Leis e Instrumentos
lei_prezeis = REC["Lei_do_PREZEIS_1995"]
lei_remembramento = REC["Lei_do_Remembramento_2020"]

# Espaços e Danos
zeis_coque = REC["ZEIS_Coque"]
dano_gentrificacao_coque = REC["Risco_de_Gentrificacao_Coque"]

# Adicionando os indivíduos às suas classes para que o sistema saiba o que eles são
g.add((prefeitura, RDF.type, REC.Estado_Inativo)) # Neste contexto, a prefeitura atuou como agente omisso/contraditório ao sancionar a lei
g.add((mercado_imobiliario_especulativo, RDF.type, REC.Elite_Proprietaria))
g.add((comunidade_coque, RDF.type, OWL.NamedIndividual)) # Apenas um indivíduo no mundo por enquanto

g.add((lei_prezeis, RDF.type, REC.InstrumentoAcao))
g.add((lei_remembramento, RDF.type, OWL.NamedIndividual)) # É uma lei, mas seu efeito é o de Veto/Inação

g.add((zeis_coque, RDF.type, REC.ZEIS))
g.add((dano_gentrificacao_coque, RDF.type, REC.DanoUrbano))


# 3. CONECTAR OS PONTOS: MODELANDO A DINÂMICA DE PODER
# -----------------------------------------------------------------------------
# Aqui está a narrativa sendo escrita com lógica.

# A Lei de Remembramento causa diretamente o risco de gentrificação.
g.add((lei_remembramento, REC.causa_direta, dano_gentrificacao_coque))

# A Lei do PREZEIS, por outro lado, foi criada para impedir a gentrificação.
g.add((lei_prezeis, REC.reverte_por_forca, dano_gentrificacao_coque))

# O Mercado Especulativo e a Comunidade têm objetivos opostos.
g.add((mercado_imobiliario_especulativo, REC.em_antagonismo_com, comunidade_coque))

# A Prefeitura, ao sancionar a nova lei, impede a eficácia da lei antiga.
g.add((prefeitura, REC.impede, lei_prezeis))


# 4. SALVAR A BASE DE CONHECIMENTO INSTANCIADA
# -----------------------------------------------------------------------------
output_path_instanciado = "../data/kb_conflito_coque_instanciado.ttl"
g.serialize(destination=output_path_instanciado, format="turtle")

print(f"Base de conhecimento com o caso do Coque foi salva em: {output_path_instanciado}")
print("\n--- Visualização de algumas triplas do conflito ---")
for s, p, o in g:
    # Mostra apenas as triplas que usam nossas propriedades de conflito
    if "recife.leg.br" in str(p):
        print(f"{s.n3(g.namespace_manager)} {p.n3(g.namespace_manager)} {o.n3(g.namespace_manager)} .")

Base de conhecimento com o caso do Coque foi salva em: ../data/kb_conflito_coque_instanciado.ttl

--- Visualização de algumas triplas do conflito ---
rec:Mercado_Imobiliario_Especulativo rec:em_antagonismo_com rec:Comunidade_do_Coque .
rec:Lei_do_PREZEIS_1995 rec:reverte_por_forca rec:Risco_de_Gentrificacao_Coque .
rec:Lei_do_Remembramento_2020 rec:causa_direta rec:Risco_de_Gentrificacao_Coque .
rec:Prefeitura_do_Recife rec:impede rec:Lei_do_PREZEIS_1995 .
